In [1]:
import requests
import time
import pprint
import pickle
import numpy as np
import json
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from wordcloud import WordCloud
import xml.etree.ElementTree as ET
import re
from unicodedata import normalize
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from matplotlib import pyplot
import seaborn as sns
from sklearn.preprocessing import Imputer
%matplotlib
%pylab inline
pylab.rcParams['figure.figsize'] = (20, 10)

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


## Importar tablas

In [2]:
tablin = pd.read_csv(r'C:\Users\Marta\Desktop\Rotación 2 SBD\BackUp_Proyecto\Python Scripts\Modelo evolución precios\Datos - copia - Marta\tablin_regresion_lineal2.csv', 
                    sep=';', engine="python", decimal=',', encoding="latin1")   #14.000 rows aprox
#Tabla que viene de la regresión lineal e incluye las predicciones

In [3]:
data_detalle = pd.read_csv(r'C:\Users\Marta\Desktop\Rotación 2 SBD\BackUp_Proyecto\Python Scripts\Proyecto\Datos\data_detalle_actualizado.csv', 
                   sep=';', encoding="latin1", engine="python", decimal=',')   #150.000 rows aprox
#Tabla que contiene muchos detalles de algunas viviendas (vs. oracle_to_test_model_baseline_XML.csv, que tiene menos detalle)

In [4]:
data_tasadoras = pd.read_csv(r'C:\Users\Marta\Desktop\Rotación 2 SBD\BackUp_Proyecto\Python Scripts\Proyecto\Datos\comparacion_tasadoras\Compraracion_tasadoras.csv', 
                   sep=';', encoding="latin1", engine="python", decimal=',')
#Tabla externa que viene de las tasadoras

In [5]:
data_XML_OSM = pd.read_csv(r'C:\Users\Marta\Desktop\Rotación 2 SBD\BackUp_Proyecto\Python Scripts\Proyecto\Datos\data_detalle_actualizado_OSM_sin_duplicados.csv', 
                   sep=';', engine="python", decimal=',', encoding='latin1')
# Tabla que viene del script de creación variable OSM

In [19]:
#with open(r'C:\Users\Marta\Desktop\Rotación 2 SBD\BackUp_Proyecto\Python Scripts\Modelo evolución precios\Pickles\dict_pc.pickle', 'rb') as file:
   # dict_pc = pickle.load(file)

EOFError: Ran out of input

## Procesar

In [6]:
data_detalle = data_detalle[data_detalle['SUPERF_ADOPTADA']>1]
data_detalle['Precio m2'] = data_detalle.loc[:, ['IMPTOTAL', 'SUPERF_ADOPTADA']].apply(lambda s: s[0]/s[1], axis=1)
data_detalle['Index'] = data_detalle.index   

In [7]:
data_pred= pd.merge(data_detalle, tablin, on=['Fecha', 'Municipio'], how='left')
data_pred = data_pred.drop_duplicates()

In [8]:
dates = {}
iter_ = 0
for year in range(2005,2019):
    for month in ['01','04','07','10']:
        for day in ['01']:
            dates[day+'/'+month+'/'+str(year)] = iter_
            iter_ += 1

In [9]:
data_pred['Fecha_num'] = data_pred['Fecha'].map(dates)

In [10]:
data_pred_index = data_pred.loc[:,['MATRICULA','Fecha_num']].groupby('MATRICULA').idxmax()   #agrupamos por matrícula (la fecha más posterior)

In [11]:
data_pred = data_pred.loc[data_pred_index['Fecha_num']]   #filtramos y nos quedamos sin duplicados de la misma vivienda con distinta fecha

In [12]:
#Recodificar
data_pred= data_pred.replace('SI', '1')
data_pred= data_pred.replace('NO', '0')
data_pred= data_pred.replace('LIBRE', 'Libre' )
data_pred= data_pred.replace('ESPECIAL', 'Especial')

## Modelo con regresión lineal y sin actualizar

In [13]:
data_pred['Predict'] = data_pred.loc[:, ['SUPERF_ADOPTADA', 'linear_regression']].apply(lambda s: s[0]*s[1], axis=1)
data_pred['Diferencia'] = data_pred.loc[:, ['IMPTOTAL', 'Predict']].apply(lambda s: abs(s[0]-s[1]), axis=1) #imptotal - predicción = diferencia
#Parte del coste no explicada por la regresión lineal
data_pred['Porcentaje'] = data_pred.loc[:, ['Diferencia', 'IMPTOTAL']].apply(lambda s: s[0]/s[1], axis=1)
#Porcentaje de la diferencia (porcentaje no explicado por la regresión lineal)
data_pred['Porcentaje'].mean() #MAPE: media de la variación porcentual . Indica que nuestras predicciones tienen una variación del 28%. Ahora del 25%

0.2755814066989645

In [14]:
data_pred['Year'] = data_pred.Fecha.apply(lambda s: s[6:])

In [15]:
data_pred = data_pred.drop(columns=['Predict', 'Diferencia', 'Porcentaje', 'parque_cat.1', 'tasa_cat.1'])

## Eliminar duplicados y nulos

In [16]:
data_pred = data_pred.drop_duplicates()
data_pred= data_pred.dropna(subset=['Provincia']) 

## Procesar y hacer un merge para obtener todas las variables (código postal, OSM, etc.) 

In [17]:
#Map postal code so it appears as a random number that identifies the actual postal code
data_XML_OSM['Codigo_Postal'] = data_XML_OSM['Municipio'].map(dict_pc)
data_XML_OSM.loc[data_XML_OSM.Codigo_Postal.isnull(),'Codigo_Postal'] = [38297]*sum(data_XML_OSM.Codigo_Postal.isnull())

NameError: name 'dict_pc' is not defined

In [ ]:
for col in ['EDIF_LICENCIA_COD','EDIF_CALIDAD_CONSTRU_COD','EDIF_EST_CONSERVACION_COD','VIV_ORIENTACION_COD','EDIF_FECHA_CONSTRUCCION']:
    data_pred.loc[:,col] = data_pred.loc[:,col].apply(lambda s: str(s))

In [ ]:
data_pred.loc[:,'EDIF_FECHA_CONSTRUCCION'] = data_pred['EDIF_FECHA_CONSTRUCCION'].apply(lambda s: (s[5:9]))

In [ ]:
#Crear variables dummies 
hi= data_pred.loc[:,['UBICACION_MUNICIPIO_DESC',
'UBICACION_ZONA_COD','UBICACION_INMUEBLE_COD','Municipio']]
dummy = pd.get_dummies(hi)
data_pred = pd.concat([data_pred, dummy], axis=1)
data_pred = data_pred.drop(['UBICACION_MUNICIPIO_DESC',
'UBICACION_ZONA_COD','UBICACION_INMUEBLE_COD','Municipio'], axis=1)
#YA TENEMOS EL DATASET "PRUEBA", PREPARADO PARA MACHINE LEARNING, CON LOS MUNICIPIOS COMO DUMMIES Y EL AÑO CON UN ÍNDICE
#HABRÍA QUE PROBAR SI SERÍA MEJOR SEGUIR HACIENDO EL LOOP POR MUNICIPIOS O DEJARLOS COMO DUMMIES
#EDIF_FECHA_CONSTRUCCION   convertirla a numérica y así se puede introducir

## Impute 

In [ ]:
data_pred['EDIF_FECHA_CONSTRUCCION']= data_pred.loc[:,['EDIF_FECHA_CONSTRUCCION']].replace("", np.nan)

In [ ]:
#data_pred = data_pred.fillna(data_pred.median())

In [ ]:
data_pred= data_pred.dropna() 

## Merge

In [ ]:
dataframe= data_pred.merge(data_XML_OSM.loc[:,['MATRICULA','Establecimientos_OSM','Codigo_Postal']], left_on=['MATRICULA'],
                                right_on=['MATRICULA'],
                                left_index=True, how='inner')
dataframe = dataframe.drop_duplicates()

## De la fecha de construcción, obtener una variable de antigüedad 

In [ ]:
#Convertimos la fecha de construccion a una variable de antigüedad de la vivienda
dataframe['EDIF_FECHA_CONSTRUCCION']= dataframe['EDIF_FECHA_CONSTRUCCION'].astype(int)
indexNames = dataframe[(dataframe['EDIF_FECHA_CONSTRUCCION'] < 1800)].index
dataframe.drop(indexNames , inplace=True)
dataframe['EDIF_FECHA_CONSTRUCCION']= dataframe['EDIF_FECHA_CONSTRUCCION'] - 1800

## Quitar outliers y guardar la tabla final

In [ ]:
#Quitar outliers
dataframe= dataframe[np.abs(dataframe['IMPTOTAL']-dataframe['IMPTOTAL'].mean()) <= (3*dataframe['IMPTOTAL'].std())] 

In [ ]:
dataframe.to_csv(r'C:\Users\Marta\Desktop\Rotación 2 SBD\BackUp_Proyecto\Python Scripts\Modelo evolución precios\Datos - copia - Marta\tabla_modelos.csv', 
                   sep=';', encoding="latin1", decimal=',', index=False)